In [1]:
import glob
import os
import librosa
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
%matplotlib inline
# plt.style.use('ggplot')

/home/tianwei/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
seed = 1234
tf.set_random_seed(seed)
np.random.seed(seed)

In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Loading model

In [4]:
# model info:

# _________________________________________________________________
# Layer (type)                 Output Shape              Param #   
# =================================================================
# conv2d_1 (Conv2D)            (None, 38, 38, 32)        896       
# _________________________________________________________________
# activation_1 (Activation)    (None, 38, 38, 32)        0         
# _________________________________________________________________
# conv2d_2 (Conv2D)            (None, 36, 36, 32)        9248      
# _________________________________________________________________
# activation_2 (Activation)    (None, 36, 36, 32)        0         
# _________________________________________________________________
# max_pooling2d_1 (MaxPooling2 (None, 18, 18, 32)        0         
# _________________________________________________________________
# dropout_1 (Dropout)          (None, 18, 18, 32)        0         
# _________________________________________________________________
# conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
# _________________________________________________________________
# activation_3 (Activation)    (None, 16, 16, 64)        0         
# _________________________________________________________________
# conv2d_4 (Conv2D)            (None, 14, 14, 64)        36928     
# _________________________________________________________________
# activation_4 (Activation)    (None, 14, 14, 64)        0         
# _________________________________________________________________
# max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
# _________________________________________________________________
# dropout_2 (Dropout)          (None, 7, 7, 64)          0         
# _________________________________________________________________
# flatten_1 (Flatten)          (None, 3136)              0         
# _________________________________________________________________
# dense_1 (Dense)              (None, 256)               803072    
# _________________________________________________________________
# activation_5 (Activation)    (None, 256)               0         
# _________________________________________________________________
# dropout_3 (Dropout)          (None, 256)               0         
# _________________________________________________________________
# dense_2 (Dense)              (None, 10)                2570      
# _________________________________________________________________
# activation_6 (Activation)    (None, 10)                0         
# =================================================================
# Total params: 871,210
# Trainable params: 871,210
# Non-trainable params: 0
# _________________________________________________________________

# 0.9M params, about 4MB (keras 10MB)

In [5]:
def exp_evidence(logits): 
    return tf.exp(tf.clip_by_value(logits,-10,10))

#### KL Divergence calculator

def KL(alpha, K):
    beta=tf.constant(np.ones((1,K)),dtype=tf.float32)
    S_alpha = tf.reduce_sum(alpha,axis=1,keepdims=True)
    
    KL = tf.reduce_sum((alpha - beta)*(tf.digamma(alpha)-tf.digamma(S_alpha)),axis=1,keepdims=True) + \
         tf.lgamma(S_alpha) - tf.reduce_sum(tf.lgamma(alpha),axis=1,keepdims=True) + \
         tf.reduce_sum(tf.lgamma(beta),axis=1,keepdims=True) - tf.lgamma(tf.reduce_sum(beta,axis=1,keepdims=True))
    return KL

def mse_loss(p, alpha, K, global_step, annealing_step): 
    S = tf.reduce_sum(alpha, axis=1, keep_dims=True) 
    E = alpha - 1
    m = alpha / S
    
    A = tf.reduce_sum((p-m)**2, axis=1, keep_dims=True) 
    B = tf.reduce_sum(alpha*(S-alpha)/(S*S*(S+1)), axis=1, keep_dims=True) 
    
    annealing_coef = tf.minimum(1.0,tf.cast(global_step/annealing_step,tf.float32))
    
    alp = E*(1-p) + 1 
    C =  annealing_coef * KL(alp, K)
    return (A + B) + C

In [6]:
# define the model
# new network:

# NETWORK PARAMETERS
data_w = 40
data_h = 40
n_classes = 10
n_filters_1 = 32
n_filters_2 = 64
d_filter = 3
p_drop_1 = 0.25
p_drop_2 = 0.50

batch_size = 256
nb_epoch = 20

K= n_classes
num_channels = 3
num_labels = n_classes

lmb = 0.00
omega = 1.0
regularizer = tf.contrib.layers.l2_regularizer(scale=omega)




X = tf.placeholder(tf.float32, shape=[None,data_w,data_h,num_channels], name = 'input')
Y = tf.placeholder(tf.float32, shape=[None,num_labels], name = 'label')

keep_prob = tf.placeholder(dtype=tf.float32, name = 'dropout_rate')
global_step = tf.Variable(initial_value=0, name='global_step', trainable=False)
annealing_step = tf.placeholder(dtype=tf.int32, name = 'annealing_step') 

### conv module

# Convolutional Layer #1
conv1 = tf.layers.conv2d(
    inputs=X,
    filters=32,
    strides=(1, 1),
    kernel_size=[3, 3],
    kernel_regularizer=regularizer,
    padding="valid"
    )
conv1_act = tf.nn.relu( conv1 )
# pool1 = tf.layers.max_pooling2d(inputs=act1, pool_size=[3, 3], strides=3)
# dropout1 = tf.layers.dropout(
#     inputs=pool1, rate=0.1)

# Convolutional Layer #2
conv2 = tf.layers.conv2d(
    inputs=conv1_act,
    filters=32,
    strides=(1, 1),
    kernel_size=[3, 3],
    kernel_regularizer=regularizer,
    padding="valid"
    )
# bn2 = tf.layers.batch_normalization(
#     conv2,
#     axis=-1
#     )
conv2_act = tf.nn.relu( conv2 )
conv2_mp = tf.layers.max_pooling2d(inputs=conv2_act, pool_size=[2, 2], strides=2)
dpout1 = tf.layers.dropout(
    inputs=conv2_mp, rate= p_drop_1)



# Convolutional Layer #3
conv3 = tf.layers.conv2d(
    inputs=dpout1,
    filters=64,
    strides=(1, 1),
    kernel_size=[3, 3],
    kernel_regularizer=regularizer,
    padding="valid"
    )
conv3_act = tf.nn.relu( conv3 )

# Convolutional Layer #4
conv4 = tf.layers.conv2d(
    inputs=conv1_act,
    filters=64,
    strides=(1, 1),
    kernel_size=[3, 3],
    kernel_regularizer=regularizer,
    padding="valid"
    )
# bn2 = tf.layers.batch_normalization(
#     conv2,
#     axis=-1
#     )
conv4_act = tf.nn.relu( conv4 )
conv4_mp = tf.layers.max_pooling2d(inputs=conv4_act, pool_size=[2, 2], strides=2)
dpout2 = tf.layers.dropout(
    inputs=conv4_mp, rate= p_drop_1)





### modify dimensions
shape = dpout2.get_shape().as_list()
flat1 = tf.reshape(dpout2, [-1, shape[1] * shape[2]* shape[3]])



### dense module

fc1 = tf.layers.dense(inputs=flat1, 
                          kernel_regularizer=regularizer,
                          units=256)
fc1_act = tf.nn.relu( fc1 )
dpout3 = tf.layers.dropout(
    inputs=fc1_act, rate= p_drop_2)

# Logits Layer
logits = tf.layers.dense(inputs=dpout3, 
                         kernel_regularizer=regularizer,
                         units=n_classes,
                        name = 'logits_tensor')
# actually name here is "logits_tensor/BiasAdd"
# because this layer is performing matmul, biasadd, (no activation)
# the cleanest way in ths future is that define the final layer as a layer with only single function.


y_ = tf.nn.softmax(logits,name="softmax_tensor")


prediction = tf.argmax(logits, 1)



########### EDL extension ###########
 
logits2evidence =  exp_evidence ############ modify this function:  relu_evidence  exp_evidence softplus

evidence = logits2evidence(logits)
alpha = evidence + 1

u = K / tf.reduce_sum(alpha, axis=1, keepdims=True)

prob = alpha/tf.reduce_sum(alpha, 1, keepdims=True) 

loss_function = mse_loss  ########### use 5th MSE loss equ: loss_eq5, loss_eq4, loss_eq3, mse_loss

loss = tf.reduce_mean(loss_function(Y, alpha, K, global_step, annealing_step))
l2_loss = tf.losses.get_regularization_loss() * lmb
loss_func = loss + l2_loss

optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss_func, global_step=global_step)

match = tf.reshape(tf.cast(tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1)), tf.float32),(-1,1))
accuracy = tf.reduce_mean(match)


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [7]:
# best_model_path = "save_model/model_test1.ckpt"
best_model_path = "SaveModel/best_model.ckpt"

saver = tf.train.Saver()
session1 =  tf.Session()
saver.restore(session1, best_model_path)
print("Model restored.")

INFO:tensorflow:Restoring parameters from SaveModel/best_model.ckpt
Model restored.


# Saving model to .pb file

In [8]:
# reference

# https://leimao.github.io/blog/Save-Load-Inference-From-TF-Frozen-Graph/
# https://github.com/leimao/Frozen_Graph_TensorFlow

In [9]:
from tensorflow.python.tools import freeze_graph

In [10]:
saver = saver
directory = 'SaveModel'
filename ='pb_model'

In [11]:
ckpt_filepath = "SaveModel/best_model.ckpt"

In [12]:
pbtxt_filename = filename + '.pbtxt'
pbtxt_filename

'pb_model.pbtxt'

In [13]:
pbtxt_filepath = os.path.join(directory, pbtxt_filename)
pb_filepath = os.path.join(directory, filename + '.pb')
print(pbtxt_filepath, pb_filepath)

SaveModel/pb_model.pbtxt SaveModel/pb_model.pb


In [14]:
tf.train.write_graph(graph_or_graph_def=session1.graph_def, logdir=directory, name=pbtxt_filename, as_text=True)


'SaveModel/pb_model.pbtxt'

In [15]:
# print(([n.name for n in tf.get_default_graph().as_graph_def().node]))

In [16]:
freeze_graph.freeze_graph(input_graph=pbtxt_filepath, 
                              input_saver='', 
                              input_binary=False, 
                              input_checkpoint=ckpt_filepath, 
                              output_node_names='logits_tensor/BiasAdd', 
                              restore_op_name='save/restore_all', 
                              filename_tensor_name='save/Const:0', 
                              output_graph=pb_filepath, 
                              clear_devices=True, 
                              initializer_nodes='')

INFO:tensorflow:Restoring parameters from SaveModel/best_model.ckpt
INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.


In [17]:
# another method?

# graph = tf.get_default_graph()
# input_graph_def = graph.as_graph_def()
# output_node_names = ['softmax_tensor', 'softmax_tensor']

# output_graph_def = graph_util.convert_variables_to_constants(self.sess, input_graph_def, output_node_names)

# with tf.gfile.GFile(pb_filepath, 'wb') as f:
#     f.write(output_graph_def.SerializeToString())


# Load .pb model

In [18]:
model_filepath = pb_filepath
print(model_filepath)

SaveModel/pb_model.pb


In [19]:
# tf.InteractiveSession.close()
'''
Lode trained model.
'''
print('Loading model...')
graph = tf.Graph()
sess = tf.InteractiveSession(graph = graph)

with tf.gfile.GFile(model_filepath, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

print('Check out the input placeholders:')
nodes = [n.name + ' => ' +  n.op for n in graph_def.node if n.op in ('Placeholder')]
for node in nodes:
    print(node)



Loading model...
Check out the input placeholders:
input => Placeholder


In [20]:
# Define input tensor
model_input = tf.placeholder(tf.float32, shape=[None,40,40,3], name = 'input')


# model_input = tf.placeholder(np.float32, shape = [None, 32, 32, 3], name='input')
# dropout_rate = tf.placeholder(tf.float32, shape = [], name = 'dropout_rate')

tf.import_graph_def(graph_def, {'input': model_input})

print('Model loading complete!')

Model loading complete!


In [21]:

# Get layer names
layers = [op.name for op in graph.get_operations()]
for layer in layers:
    print(layer)

# Check out the weights of the nodes
weight_nodes = [n for n in graph_def.node if n.op == 'Const']
for n in weight_nodes:
    print("Name of the node - %s" % n.name)
    print("Value - " )
    print(tensor_util.MakeNdarray(n.attr['value'].tensor))


input
import/input
import/conv2d/kernel
import/conv2d/kernel/read
import/conv2d/bias
import/conv2d/bias/read
import/conv2d/Conv2D
import/conv2d/BiasAdd
import/Relu
import/conv2d_3/kernel
import/conv2d_3/kernel/read
import/conv2d_3/bias
import/conv2d_3/bias/read
import/conv2d_3/Conv2D
import/conv2d_3/BiasAdd
import/Relu_3
import/max_pooling2d_1/MaxPool
import/dropout_1/Identity
import/Reshape/shape
import/Reshape
import/dense/kernel
import/dense/kernel/read
import/dense/bias
import/dense/bias/read
import/dense/MatMul
import/dense/BiasAdd
import/Relu_4
import/dropout_2/Identity
import/logits_tensor/kernel
import/logits_tensor/kernel/read
import/logits_tensor/bias
import/logits_tensor/bias/read
import/logits_tensor/MatMul
import/logits_tensor/BiasAdd
Name of the node - conv2d/kernel
Value - 


NameError: name 'tensor_util' is not defined

In [22]:
# Know your output node name
output_tensor = graph.get_tensor_by_name("import/logits_tensor/BiasAdd:0")
# output = sess.run(output_tensor, feed_dict =  {model_input:  X_test[0:2] })

# Comparing results of 2 ways

In [23]:
def one_hot_encode(labels, num_class):
    n_labels = len(labels)
#     n_unique_labels = len(np.unique(labels))
    n_unique_labels = num_class
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [24]:
## GET DATA TO WORK ON
print("Start loading data")

fd = open("data_x.pkl", 'rb')
fd2 = open("data_y.pkl", 'rb')
features = pickle.load(fd)
labels = pickle.load(fd2)

print("Data loaded")

# all testing data
X_test = features[:]
Y_test = labels[:]

X_test = X_test.astype('float32')
X_test /= 255

## one hot encoding
Y_test = one_hot_encode(Y_test, 10)

print(X_test.shape)
print(Y_test.shape)

batch_size = 256

n_batches_test = Y_test.shape[0]//batch_size
print('For batch of size %d: \n %d batches in test'%(batch_size, n_batches_test))

Start loading data
Data loaded
(54154, 40, 40, 3)
(54154, 10)
For batch of size 256: 
 211 batches in test


# pb model output

In [25]:
pb_output = sess.run(output_tensor, feed_dict =  {model_input:  X_test[0:2] })
pb_output

array([[ 10.591967 , -40.03761  , -11.062835 ,  -9.509928 , -25.767548 ,
        -11.834668 , -22.799807 , -22.827818 ,  -5.7495046, -34.31049  ],
       [ 17.028324 , -40.26766  ,  -7.9120092, -14.365061 , -26.771418 ,
        -15.199751 , -27.301739 , -26.882553 ,  -8.996705 , -33.614563 ]],
      dtype=float32)

In [26]:
def predict_label(logits_output):
    predict_y = np.argmax(logits_output, axis = 1)
    return predict_y

def exp_evidence(logits): 
    return np.exp(np.clip(logits,-10,10))

def uncertainty_score( logits_output ):
    evidence = exp_evidence(logits_output)
    alpha = evidence + 1
    u_score = 10 / np.sum(alpha, axis=1, keepdims=True)  # K = num_classes = 10
    return u_score

print(uncertainty_score(pb_output))

print(predict_label(pb_output))

[[0.00045379]
 [0.00045379]]
[0 0]


In [27]:
y_pred, u, logits = session1.run([prediction, u, logits], feed_dict={X: X_test[0:2], Y : Y_test[0:2], keep_prob:1.,  annealing_step:100*n_batches_test})

print(y_pred,u)
print(logits)

[0 0] [[0.00045379]
 [0.00045379]]
[[ 10.591967  -40.03761   -11.062835   -9.509928  -25.767548  -11.834668
  -22.799807  -22.827818   -5.7495046 -34.31049  ]
 [ 17.028324  -40.26766    -7.9120092 -14.365061  -26.771418  -15.199751
  -27.301739  -26.882553   -8.996705  -33.614563 ]]


In [28]:
# About classes:
# 0	air_conditioner
# 1	car_horn
# 2	children_playing
# 3	dog_bark
# 4	drilling
# 5	engine_idling
# 6	gun_shot
# 7	jackhammer
# 8	siren
# 9	street_music